In [1]:
%load_ext autoreload
%autoreload 2
import wallaroo

import os, json
os.environ['DEPLOYMENT_CONFIG'] = json.dumps({"cpus": 0.5, "replica_count": 1, "memory": "500Mi"})

auth_type = "sso"
client = wallaroo.Client(auth_type=auth_type, request_timeout=90, interactive=True)

In [2]:
for p in client.list_pipelines(): p.undeploy()

 ok
Waiting for undeployment - this will take up to 90s ........... ok


In [3]:
import pyarrow as pa
input_schema = pa.schema([
    pa.field('dense_2', pa.list_(pa.float64()))
])
output_schema = pa.schema([
    pa.field('output', pa.list_(pa.float64()))
])

In [4]:
from wallaroo.framework import Framework
step = client.upload_model("python-step", "./step.py", 
                           framework=Framework.PYTHON).configure('python', 
                                                                 input_schema=input_schema, 
                                                                 output_schema=output_schema)


In [5]:
pipeline = client.build_pipeline("py-step-test")
pipeline = pipeline.add_model_step(step)
deployment = pipeline.deploy()

Waiting for deployment - this will take up to 90s ................. ok


In [6]:
import pandas as pd
data = pd.DataFrame.from_dict({"dense_2": [[0.6878518042239091,
 0.17607340208535074,
 -0.8695140830357148,
 0.34638762962802144,
 -0.0916270832672289,
 -0.022063226781124278,
 -0.13969884765926363,
 1.002792335666138,
 -0.3067449033633758,
 0.9272000630461978,
 0.28326687982544635,
 0.35935375728372815,
 -0.682562654045523,
 0.532642794275658,
 -0.22705189652659302,
 0.5743846356405602,
 -0.18805086358065454]]})
deployment.infer(data)

,time,in.dense_2,out.output,check_failures
0,2023-06-22 17:28:52.072,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[0.6878518042],0


In [7]:
deployment.logs()

,time,in.dense_2,out.output,check_failures
0,2023-06-22 17:28:52.072,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[0.6878518042],0
1,2023-06-20 20:20:50.418,"[0.6878518042, 0.1760734021, -0.869514083, 0.3...",[0.6878518042],0


In [9]:
deployment.undeploy()


Waiting for undeployment - this will take up to 90s ....................................... ok


name,py-step-test
created,2023-06-20 20:04:46.758046+00:00
last_updated,2023-06-22 17:28:32.826710+00:00
deployed,False
tags,
versions,"d48348ff-da70-43c8-b564-14b2cff8ed51, 02bef6cf-edb3-4718-8558-20e0a2fb6f46, 8660e556-b795-444d-b53e-aeb031a077d8, d00827f8-3362-43d3-854a-485a7070cb3b, 0d80a882-f1a9-4fab-863e-631b753e0830, 0dc0b38e-140d-4d92-9f75-0b8e0fb8088a, c8b1b2ce-c722-4511-b75e-20399e94c10e, 047674d6-6c4c-43a8-900c-f5d4c2a4abf6, c1451a4a-5671-4686-8f05-afb60f6bcd15, 594357ea-7d32-4528-99be-300af2a06071, 52dfb80e-c199-4451-9b0a-7209ad345e50, 0c7bcd0f-dd09-4754-8203-615d47038a99, 330a3695-f2e6-4218-8636-3d66820fb7b4, 2298f0ec-4cfe-4668-b392-901fb6a97292"
steps,python-step
